In [23]:
# 시계열 데이터 Time Series Data 분석
# 이동평균 계산
# 변화율 계산
# 센서 데이터 이상치 탐지
# OEE 계산하는 방법(공장, 생산 설비가 얼마나 효율적으로 운영되는지 측정하는 KPI)

In [24]:
import pandas as pd
import numpy as np

In [25]:
production_df = pd.read_csv('./data/05_production.csv', encoding='utf-8-sig')
quality_df = pd.read_csv('./data/07_quality_inspection.csv', encoding='utf-8-sig', na_values=['\\N'])
sensor_df = pd.read_csv('./data/08_sensor_data.csv', encoding='utf-8-sig')
operation_df = pd.read_csv('./data/06_equipment_operation.csv', encoding='utf-8-sig')
equipment_df = pd.read_csv('./data/01_equipment.csv', encoding='utf-8-sig')

# 날짜/시간 변환
production_df['production_date'] = pd.to_datetime(production_df['production_date'])
production_df['start_time'] = pd.to_datetime(production_df['start_time'])
production_df['end_time'] = pd.to_datetime(production_df['end_time'])
sensor_df['measurement_time'] = pd.to_datetime(sensor_df['measurement_time'])
operation_df['start_time'] = pd.to_datetime(operation_df['start_time'])
operation_df['end_time'] = pd.to_datetime(operation_df['end_time'])

In [26]:
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48


In [27]:
# 일별 새산량 추이 -> 일별로 집계
daily_production = production_df.groupby('production_date').agg({'production_id':'count',
                                              'actual_quantity':'sum',
                                              'defect_quantity':'sum'})
daily_production.columns = ['생산 건수', '총 생산량', '총 불량수']
daily_production

,생산 건수,총 생산량,총 불량수
production_date,,,
2024-01-01,20,2019,114
2024-01-02,22,2380,128
2024-01-03,16,1848,95
2024-01-04,22,2358,128
2024-01-05,20,2330,123
...,...,...,...
2024-03-27,22,2496,386
2024-03-28,20,2107,310
2024-03-29,22,2551,391


In [28]:
daily_production['불량률'] = (daily_production['총 불량수']/daily_production['총 생산량']*100).round(2)
daily_production

,생산 건수,총 생산량,총 불량수,불량률
production_date,,,,
2024-01-01,20,2019,114,5.65
2024-01-02,22,2380,128,5.38
2024-01-03,16,1848,95,5.14
2024-01-04,22,2358,128,5.43
2024-01-05,20,2330,123,5.28
...,...,...,...,...
2024-03-27,22,2496,386,15.46
2024-03-28,20,2107,310,14.71
2024-03-29,22,2551,391,15.33


In [29]:
sensor_df

,sensor_id,equipment_id,measurement_time,temperature,pressure,vibration,current,voltage,rpm,created_at
0,1,INJ-001,2024-01-01 00:00:00,183.93,148.65,2.6838,48.05,218.83,1795.32,2026-01-30 00:45:52
1,2,INJ-002,2024-01-01 00:00:00,173.06,144.23,2.2576,39.17,211.38,1738.75,2026-01-30 00:45:52
2,3,PRESS-001,2024-01-01 00:00:00,82.29,202.09,3.4924,120.14,372.88,489.11,2026-01-30 00:45:52
3,4,PRESS-002,2024-01-01 00:00:00,87.62,194.68,3.7005,121.70,379.93,498.85,2026-01-30 00:45:52
4,5,ASM-001,2024-01-01 00:00:00,20.78,-0.94,1.1674,16.48,220.86,97.69,2026-01-30 00:45:52
...,...,...,...,...,...,...,...,...,...,...
10915,10916,INJ-001,2024-03-31 23:00:00,192.48,148.93,2.4576,48.58,226.34,1775.78,2026-01-30 00:45:52
10916,10917,INJ-002,2024-03-31 23:00:00,175.82,141.36,2.4609,44.68,226.68,1762.20,2026-01-30 00:45:52
10917,10918,PRESS-001,2024-03-31 23:00:00,84.57,191.96,3.5106,116.58,383.60,492.28,2026-01-30 00:45:52
10918,10919,PRESS-002,2024-03-31 23:00:00,87.53,202.47,3.8016,118.02,390.56,506.57,2026-01-30 00:45:52


In [36]:
# Method 1 - 설비별 시계열 데이터 분석
sensor_1 = sensor_df[sensor_df['equipment_id'] == 'INJ-001'].sort_values('measurement_time')
sensor_1.reset_index(drop=True, inplace=True)
sensor_1

,sensor_id,equipment_id,measurement_time,temperature,pressure,vibration,current,voltage,rpm,created_at
0,1,INJ-001,2024-01-01 00:00:00,183.93,148.65,2.6838,48.05,218.83,1795.32,2026-01-30 00:45:52
1,6,INJ-001,2024-01-01 01:00:00,179.26,155.23,2.6619,41.47,221.62,1792.30,2026-01-30 00:45:52
2,11,INJ-001,2024-01-01 02:00:00,179.78,146.19,2.3795,42.07,221.48,1805.22,2026-01-30 00:45:52
3,16,INJ-001,2024-01-01 03:00:00,181.30,143.83,2.4134,47.61,211.96,1803.69,2026-01-30 00:45:52
4,21,INJ-001,2024-01-01 04:00:00,179.96,152.57,2.4701,44.85,215.77,1769.70,2026-01-30 00:45:52
...,...,...,...,...,...,...,...,...,...,...
2179,10896,INJ-001,2024-03-31 19:00:00,194.25,152.07,2.5405,45.38,215.45,1817.42,2026-01-30 00:45:52
2180,10901,INJ-001,2024-03-31 20:00:00,192.63,145.85,2.4024,46.84,222.41,1773.20,2026-01-30 00:45:52
2181,10906,INJ-001,2024-03-31 21:00:00,196.04,149.04,2.5558,41.28,227.03,1830.42,2026-01-30 00:45:52
2182,10911,INJ-001,2024-03-31 22:00:00,194.29,145.70,2.4921,43.75,216.40,1778.40,2026-01-30 00:45:52


In [43]:
# 일단위 평균
# resample 함수를 이용해서 datetime을 조절할 때는 datetime column이 index로 되어 있어야 함
sensor_1.set_index('measurement_time').loc[:, 'temperature':'rpm'].resample('D').mean()

,temperature,pressure,vibration,current,voltage,rpm
measurement_time,,,,,,
2024-01-01,180.656250,149.338750,2.499879,44.680417,221.020000,1801.248333
2024-01-02,179.808333,149.557083,2.453850,44.813333,221.112917,1803.249167
2024-01-03,180.512083,151.159583,2.553117,45.706250,220.699583,1800.577500
2024-01-04,181.021667,149.677083,2.460775,44.397083,218.150417,1800.027917
2024-01-05,180.211667,150.191667,2.487862,45.253750,219.357917,1799.818333
...,...,...,...,...,...,...
2024-03-27,193.929583,150.450833,2.500167,45.898333,220.992917,1799.215833
2024-03-28,193.542083,150.111667,2.467542,45.199583,220.824167,1804.017917
2024-03-29,194.460417,150.050000,2.516758,44.872083,220.793333,1799.270000


In [47]:
sensor_all = sensor_df.groupby('measurement_time')[['temperature', 'pressure', 'vibration', 'current', 'voltage', 'rpm']].mean()
sensor_all

,temperature,pressure,vibration,current,voltage,rpm
measurement_time,,,,,,
2024-01-01 00:00:00,109.536,137.742,2.66034,69.108,280.776,923.944
2024-01-01 01:00:00,109.550,140.314,3.33218,68.274,283.618,923.094
2024-01-01 02:00:00,110.452,137.004,2.75904,66.716,287.544,940.688
2024-01-01 03:00:00,111.628,138.982,2.78806,68.062,284.330,939.540
2024-01-01 04:00:00,111.246,140.722,2.60828,68.554,284.042,924.738
...,...,...,...,...,...,...
2024-03-31 19:00:00,113.852,136.950,2.76472,67.746,282.042,937.338
2024-03-31 20:00:00,112.890,139.950,3.10002,68.502,281.266,930.456
2024-03-31 21:00:00,114.270,138.820,2.60134,68.698,287.960,953.128


In [48]:
sensor_all.resample('D').mean()

,temperature,pressure,vibration,current,voltage,rpm
measurement_time,,,,,,
2024-01-01,110.660667,139.611750,2.814983,68.186667,284.601583,932.055000
2024-01-02,111.031417,139.831500,2.686983,68.438750,284.228583,936.736667
2024-01-03,110.668000,140.153833,2.695356,68.191667,284.183250,932.576417
2024-01-04,110.718333,139.901167,2.712816,68.124250,284.065250,934.209417
2024-01-05,110.373833,139.653000,2.751715,68.151167,283.345417,934.190083
...,...,...,...,...,...,...
2024-03-27,113.264417,139.593083,2.667431,68.441833,285.089500,933.713583
2024-03-28,113.161583,139.867250,2.704023,68.069917,284.704333,935.672583
2024-03-29,113.754250,139.826000,2.767098,68.518333,284.464333,932.220750


In [51]:
# 생산 효율 = target_quantity/actual_quantity
production_df['생산 효율'] = (production_df['actual_quantity']/production_df['target_quantity']*100).round(2)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,생산 효율
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,83.51
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,93.98
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,90.60
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,92.00
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,104.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,96.00
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.59
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.24
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,84.62


In [55]:
def get_grade(x):
    if x > 90: return '우수'
    elif x > 80: return '양호'
    else: return '부족'
production_df['효율 등급'] = production_df['생산 효율'].apply(get_grade)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,생산 효율,효율 등급
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,83.51,양호
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,93.98,우수
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,90.60,우수
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,92.00,우수
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,104.88,우수
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,96.00,우수
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.59,우수
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.24,우수
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,84.62,양호


In [57]:
# 이동 평균
# 노이즈 제거 및 이상 패턴 감지

# 진동센서 7일 이동평균 상승 -> 베어링 교체
# 온도 30일 이동평균 상승 -> 냉각 시스템 점검
# 측정값 이동평균이 상한선에 접근 -> 사전 경고
# 생산량 이동평균으로 수요 예측
# 계절성 패턴 파악

In [61]:
# INJ-001의 온도 데이터 이동 평균
sensor_1.set_index('measurement_time')['temperature'].rolling(window=7).mean() # 7시간
sensor_1.set_index('measurement_time').resample('D')['temperature'].mean().rolling(window=7).mean() # 7일

measurement_time
2024-01-01           NaN
2024-01-02           NaN
2024-01-03           NaN
2024-01-04           NaN
2024-01-05           NaN
                 ...    
2024-03-27    193.697202
2024-03-28    193.754643
2024-03-29    193.935536
2024-03-30    194.142917
2024-03-31    194.485833
Freq: D, Name: temperature, Length: 91, dtype: float64

In [68]:
# 전일 대비 증감량
# daily_production - daily_production.shift(1)
daily_production.diff().round(2)

,생산 건수,총 생산량,총 불량수,불량률
production_date,,,,
2024-01-01,NaN,NaN,NaN,NaN
2024-01-02,2.0,361.0,14.0,-0.27
2024-01-03,-6.0,-532.0,-33.0,-0.24
2024-01-04,6.0,510.0,33.0,0.29
2024-01-05,-2.0,-28.0,-5.0,-0.15
...,...,...,...,...
2024-03-27,4.0,364.0,82.0,1.20
2024-03-28,-2.0,-389.0,-76.0,-0.75
2024-03-29,2.0,444.0,81.0,0.62


In [67]:
# 변화율 계산
(daily_production.pct_change()*100).round(2)

,생산 건수,총 생산량,총 불량수,불량률
production_date,,,,
2024-01-01,NaN,NaN,NaN,NaN
2024-01-02,10.00,17.88,12.28,-4.78
2024-01-03,-27.27,-22.35,-25.78,-4.46
2024-01-04,37.50,27.60,34.74,5.64
2024-01-05,-9.09,-1.19,-3.91,-2.76
...,...,...,...,...
2024-03-27,22.22,17.07,26.97,8.42
2024-03-28,-9.09,-15.58,-19.69,-4.85
2024-03-29,10.00,21.07,26.13,4.21


In [69]:
# 이상치 탐지
# IQR
# Z-score
# 3-Sigma

In [72]:
# OEE(Overall Equipment Effectiveness)
# 가동률 * 성능률 * 양품률
# 가동률 = 실제 가동 시간 / 계획 가동 시간
# 성능률 = (실제 생산량 * 이상 사이클 타임) / 실제 가동 시간
# 양품률 = 양품 수량 / 총 생산량

# e.g.) OEE 50% 미만 -> 현재 설비 효율 개선 필요
# e.g.) OEE 85% 이상: 설비 증설 검토

# 가동률 낮음 -> 고장, 정지 시간 줄여야 함
# 성능률 낮음 -> 속도 개선, 교체 시간 단축해야 함
# 양품률 낮음 -> 품질 개선, 재작업 줄여야 함

# 벤치마킹 가능 -> 설비간 비교를 통해 우수 설비 선정, 설비 구매엥 활용
# 공장간 비교 가능 -> 우수 공장을 Best Practice로 선정

In [73]:
# OEE 계산 결과
# 85% 이상: 월클
# 70 ~ 84%: 양호
# 60 ~ 69%: 보통, 개선 과제 도출
# 60% 미만: 개선 필요, 긴급 조치

In [76]:
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,생산 효율,효율 등급,불량률
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,83.51,양호,4.94
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,93.98,우수,7.69
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,90.60,우수,2.22
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,92.00,우수,2.17
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,104.88,우수,5.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,96.00,우수,17.36
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.59,우수,16.15
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.24,우수,17.50
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,84.62,양호,16.53


In [77]:
production_df['불량률'] = (production_df['defect_quantity']/production_df['actual_quantity']*100).round(2)
production_df

,production_id,equipment_id,product_code,production_date,start_time,end_time,target_quantity,actual_quantity,good_quantity,defect_quantity,cycle_time,work_order_no,lot_no,operator_id,shift,created_at,updated_at,생산 효율,효율 등급,불량률
0,1,INJ-001,BUMPER-A,2024-01-01,2024-01-01 08:14:00,2024-01-01 09:53:32,97,81,77,4,73.73,WO202401019935,LOT2024010100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,83.51,양호,4.94
1,2,INJ-001,BUMPER-A,2024-01-01,2024-01-01 21:02:00,2024-01-01 22:33:43,83,78,72,6,70.56,WO202401012535,LOT2024010100110,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,93.98,우수,7.69
2,3,INJ-002,BUMPER-A,2024-01-01,2024-01-01 10:12:00,2024-01-01 13:16:28,149,135,132,3,81.99,WO202401018359,LOT2024010100201,OP001,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,90.60,우수,2.22
3,4,INJ-002,DASH-C,2024-01-01,2024-01-01 12:48:00,2024-01-01 15:16:31,100,92,90,2,96.87,WO202401016574,LOT2024010100202,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,92.00,우수,2.17
4,5,INJ-002,DOOR-B,2024-01-01,2024-01-01 20:48:00,2024-01-01 23:12:13,123,129,122,7,67.08,WO202401012674,LOT2024010100210,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,104.88,우수,5.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1867,1868,PRESS-002,BUMPER-A,2024-03-31,2024-03-31 20:19:00,2024-03-31 23:25:19,150,144,119,25,77.63,WO202403317101,LOT2024033100210,OP006,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,96.00,우수,17.36
1868,1869,PRESS-002,DASH-C,2024-03-31,2024-04-01 00:15:00,2024-04-01 02:59:58,136,130,109,21,76.15,WO202403318434,LOT2024033100211,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.59,우수,16.15
1869,1870,PRESS-002,BUMPER-A,2024-03-31,2024-04-01 05:53:00,2024-04-01 07:26:15,84,80,66,14,69.95,WO202403317294,LOT2024033100212,OP004,NIGHT,2026-01-30 00:42:48,2026-01-30 00:42:48,95.24,우수,17.50
1870,1871,ASM-001,BUMPER-A,2024-03-31,2024-03-31 10:24:00,2024-03-31 13:25:41,143,121,101,20,90.10,WO202403317268,LOT2024033100101,OP003,DAY,2026-01-30 00:42:48,2026-01-30 00:42:48,84.62,양호,16.53


In [80]:
# 상관분석: 두 변수 간에 어떤 선형적 관계를 갖고 있는지를 분석하는 방법
# 피어슨, 스피어만
production_df[['target_quantity', 'actual_quantity', 'cycle_time', '불량률']].corr()

,target_quantity,actual_quantity,cycle_time,불량률
target_quantity,1.000000,0.930913,0.016590,-0.075702
actual_quantity,0.930913,1.000000,-0.085386,-0.117927
cycle_time,0.016590,-0.085386,1.000000,0.058181
불량률,-0.075702,-0.117927,0.058181,1.000000
